In [20]:
from lxml import html
import requests
import re
import csv
import time 
import pandas as pd 
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

def html_parse_tree(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    return tree

def xpath_parse(tree, xpath):
    result = tree.xpath(xpath)
    return result

def regex_strip_array(array):
    for i in range(0, len(array)):
        array[i] = regex_strip_string(array[i]).strip()
    return array

def regex_strip_string(string):
    string = re.sub('\n', '', string).strip()
    string = re.sub('\r', '', string).strip()
    string = re.sub('\t', '', string).strip()
    return string

def format_spacing(max_spacing, variable):
    spacing_count = max_spacing - len(variable)
    output = ''
    for i in range(0, spacing_count):
        output += ' '
    return output

def fraction_stats(string):
    string = string.replace('(', '')
    string = string.replace(')', '')
    return string.split('/')

def add2csv(array, filename):
    with open(filename, 'a') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(array)

def array2csv(array, filename):
    with open(filename, "w+") as my_csv:
        csvWriter = csv.writer(my_csv, delimiter = ',')
        csvWriter.writerows(array)

In [23]:
year_url = "http://www.atptour.com/en/scores/results-archive?year=2022"
url_prefix = "http://www.atptour.com"

# HTML tree

options = ChromeOptions()
options.add_argument("--headless=new")
options.page_load_strategy = 'normal' 
chrome_path = "./bin/chromedriver.exe" #ChromeDriverManager()#.install() 
chrome_service = Service(chrome_path) 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)
#atpurl = "https://www.atptour.com/en/scores/current/us-open/560/daily-schedule"
driver.get(year_url) 


year_tree = html.fromstring(driver.page_source)  #html_parse_tree(year_url)

    # Initial XPath to find number of tournaments in a given year
tourney_titles_xpath = "//span[contains(@class, 'tourney-title')]/text()"
tourney_titles_parsed = xpath_parse(year_tree, tourney_titles_xpath)
tourney_titles_cleaned = regex_strip_array(tourney_titles_parsed)

# If no tournaments found in <span> tags try find in <a> tags
if len(tourney_titles_cleaned) == 0:
    tourney_titles_xpath = "//a[contains(@class, 'tourney-title')]/text()"
    tourney_titles_parsed = xpath_parse(year_tree, tourney_titles_xpath)
    tourney_titles_cleaned = regex_strip_array(tourney_titles_parsed)

print(tourney_titles_cleaned)

tourney_count = len(tourney_titles_cleaned)


['ATP Cup', 'Adelaide 1', 'Melbourne', 'Sydney', 'Adelaide 2', 'Australian Open', 'Cordoba', 'Pune', 'Montpellier', 'Rotterdam', 'Dallas', 'Buenos Aires', 'Rio de Janeiro', 'Doha', 'Delray Beach', 'Marseille', 'Dubai', 'Acapulco', 'Santiago', 'ATP Masters 1000 Indian Wells', 'ATP Masters 1000 Miami', 'Houston', 'Marrakech', 'ATP Masters 1000 Monte Carlo', 'Barcelona', 'Belgrade', 'Estoril', 'Munich', 'ATP Masters 1000 Madrid', 'ATP Masters 1000 Rome', 'Geneva', 'Lyon', 'Roland Garros', 'Stuttgart', "'s-Hertogenbosch", 'Halle', "London / Queen's Club", 'Mallorca', 'Eastbourne', 'Wimbledon', 'Newport', 'Bastad', 'Hamburg', 'Gstaad', 'Atlanta', 'Kitzbuhel', 'Umag', 'Washington', 'Los Cabos', 'ATP Masters 1000 Canada', 'ATP Masters 1000 Cincinnati', 'Winston-Salem', 'US Open', 'San Diego', 'Metz', 'Laver Cup', 'Seoul', 'Tel Aviv', 'Sofia', 'Tokyo', 'Astana', 'Gijon', 'Florence', 'Antwerp', 'Stockholm', 'Naples', 'Vienna', 'Basel', 'ATP Masters 1000 Paris', 'Intesa Sanpaolo Next Gen ATP Fin

In [52]:
import pandas as pd 
import numpy as np
import math
matchs = pd.read_csv("./data/matchs.csv")


#for index, match in matchs.iterrows():
 #   print(match.start_date)

#matchs.loc[matchs.startdate.isna]

match = matchs.iloc[400]
str(match.start_date) == 'nan'



False

In [35]:
import pandas as pd 
import numpy as np
import math
matchs = pd.read_csv("./data_scrapped/matchs.csv")

#"/en/players/jimmy-connors/c044/overview",
matchs.loc[(matchs.winner_slug == "roger-federer")|(matchs.loser_slug == "roger-federer")]



#match = matchs.iloc[0]
#match.winner_slug


,tourney_year_id,tourney_order,tourney_name,tourney_slug,tourney_url_suffix,start_date,start_year,start_month,start_day,end_date,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
69,2000-7308,3,Adelaide,adelaide,/en/scores/archive/adelaide/7308/2000/results,2000.01.03,2000.0,1.0,3.0,2000.01.09,...,NaN,76(5) 64,2.0,0.0,13.0,10.0,1.0,0.0,2000-7308-2000-2-8-e113-f324,/en/scores/match-stats/archive/2000/7308/ms008
86,2000-7308,3,Adelaide,adelaide,/en/scores/archive/adelaide/7308/2000/results,2000.01.03,2000.0,1.0,3.0,2000.01.09,...,NaN,61 64,2.0,0.0,12.0,5.0,0.0,0.0,2000-7308-2000-1-7-f324-k260,/en/scores/match-stats/archive/2000/7308/ms017
114,2000-301,4,Auckland,auckland,/en/scores/archive/auckland/301/2000/results,2000.01.10,2000.0,1.0,10.0,2000.01.16,...,NaN,64 64,2.0,0.0,12.0,8.0,0.0,0.0,2000-301-2000-1-10-f316-f324,/en/scores/match-stats/archive/2000/301/ms019
178,2000-580,6,Australian Open,australian-open,/en/scores/archive/australian-open/580/2000/re...,2000.01.17,2000.0,1.0,17.0,2000.01.30,...,NaN,61 64 63,3.0,0.0,18.0,8.0,0.0,0.0,2000-580-2000-3-8-c487-f324,/en/scores/match-stats/archive/2000/580/ms028
203,2000-580,6,Australian Open,australian-open,/en/scores/archive/australian-open/580/2000/re...,2000.01.17,2000.0,1.0,17.0,2000.01.30,...,NaN,76(1) 62 63,3.0,0.0,19.0,11.0,1.0,0.0,2000-580-2000-2-15-f324-k296,/en/scores/match-stats/archive/2000/580/ms057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80981,2021-540,44,Wimbledon,wimbledon,/en/scores/archive/wimbledon/540/2021/results,2021.06.28,2021.0,6.0,28.0,2021.07.11,...,6,63 76(4) 60,3.0,0.0,19.0,9.0,1.0,0.0,2021-540-2021-8-3-hb71-f324,/en/scores/match-stats/archive/2021/540/ms007
80988,2021-540,44,Wimbledon,wimbledon,/en/scores/archive/wimbledon/540/2021/results,2021.06.28,2021.0,6.0,28.0,2021.07.11,...,23,75 64 62,3.0,0.0,19.0,11.0,0.0,0.0,2021-540-2021-7-4-f324-su87,/en/scores/match-stats/archive/2021/540/ms014
80996,2021-540,44,Wimbledon,wimbledon,/en/scores/archive/wimbledon/540/2021/results,2021.06.28,2021.0,6.0,28.0,2021.07.11,...,29,64 64 57 64,3.0,1.0,23.0,19.0,0.0,0.0,2021-540-2021-6-12-f324-n771,/en/scores/match-stats/archive/2021/540/ms028
81012,2021-540,44,Wimbledon,wimbledon,/en/scores/archive/wimbledon/540/2021/results,2021.06.28,2021.0,6.0,28.0,2021.07.11,...,NaN,76(1) 61 64,3.0,0.0,19.0,11.0,1.0,0.0,2021-540-2021-5-28-f324-g628,/en/scores/match-stats/archive/2021/540/ms056
